In [226]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim

In [188]:
train_dir = "handwritingcharacters/handwritten-english-characters-and-digits/combined_folder/train"
test_dir = "handwritingcharacters/handwritten-english-characters-and-digits/combined_folder/test"

In [189]:
df = pd.DataFrame()
df["filepath"] = ""
df["label"] = ""

In [190]:
i=0
for x in os.listdir(train_dir):
    for y in os.listdir(os.path.join(train_dir, x)):
        df.loc[i,"filepath"] = f"{train_dir}/{x}/{y}"
        df.loc[i,"label"] = x
        i+=1

In [191]:
labelEncoder = LabelEncoder()
df["label"] = labelEncoder.fit_transform(df["label"])

class_dict = {}
for i, label in enumerate(labelEncoder.classes_):
    class_dict[i] = label

In [193]:
df.to_csv("imageannotation.csv")

In [194]:
import math
train_num = math.ceil(len(df) * 0.8)
train_df = df[:train_num]
test_df = df[train_num:]
train_df.to_csv("train.csv")
test_df.to_csv("test.csv")

In [195]:
train_df["filepath"][0]

'handwritingcharacters/handwritten-english-characters-and-digits/combined_folder/train/0/0.001.png'

In [196]:
transform_to_apply = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

In [197]:
class CustomDataset(Dataset):
    def __init__(self, file, transform):
        self.df = pd.read_csv(file)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        filename = self.df.iloc[index]["filepath"]
        label = self.df.iloc[index]["label"]
        img = Image.open(filename)
        img = self.transform(img)
        label = torch.tensor(label, dtype=torch.long)
        
        return img, label
    

In [198]:
torch.manual_seed(42)

In [256]:
train_data = CustomDataset(file="imageannotation.csv", transform = transform_to_apply)
test_data = CustomDataset(file="test.csv", transform=transform_to_apply)

In [257]:
train_loader = DataLoader(train_data, batch_size=32,shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [258]:
next(iter(train_loader))[0].shape

torch.Size([32, 1, 64, 64])

In [259]:
class CustomCNN(nn.Module):
    def __init__(self, num_features, out_features):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(num_features, 32, kernel_size=3, padding="same"),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=3,stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding="same"),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3,stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(14400,128),
            nn.ReLU(),
            nn.Dropout(p=0.4),
    
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(64,out_features),
        )
    
    def forward(self, x):
        return self.classifier(self.features(x))

In [260]:
model = CustomCNN(1, len(class_dict))

In [261]:
lr = 0.01
epochs=20

In [262]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

In [263]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for img, label in train_loader:
        img = img.to(device)
        label = label.to(device).long()

        y_pred = model(img)
        loss = loss_fn(y_pred, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")


Epoch 1/20 | Loss: 5.2555
Epoch 2/20 | Loss: 4.1575
Epoch 3/20 | Loss: 4.1362
Epoch 4/20 | Loss: 4.1333
Epoch 5/20 | Loss: 4.1372
Epoch 6/20 | Loss: 4.1496
Epoch 7/20 | Loss: 4.1353
Epoch 8/20 | Loss: 4.1323
Epoch 9/20 | Loss: 4.1324
Epoch 10/20 | Loss: 4.1325
Epoch 11/20 | Loss: 4.1325
Epoch 12/20 | Loss: 4.1321
Epoch 13/20 | Loss: 4.1329
Epoch 14/20 | Loss: 4.1323
Epoch 15/20 | Loss: 4.1422
Epoch 16/20 | Loss: 4.1346
Epoch 17/20 | Loss: 4.1333
Epoch 18/20 | Loss: 4.1323
Epoch 19/20 | Loss: 4.1328
Epoch 20/20 | Loss: 4.1326


In [264]:
import torch

# Make sure model is in evaluation mode
model.eval()  

correct = 0
total = 0

with torch.no_grad():  # no gradients needed during evaluation
    for images, labels in train_loader:  # or test_loader
        images = images.to(device)       # move to GPU if using one
        labels = labels.to(device)

        outputs = model(images)          # forward pass
        _, predicted = torch.max(outputs, 1)  # get index of max probability
        total += labels.size(0)          # number of images in batch
        correct += (predicted == labels).sum().item()  # count correct predictions

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 1.61%
